In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../data/pivot_data_hourly.parquet')
# Check missing values
missing = df.isna().sum()
print(missing[missing > 0])

# Summary statistics
print(df.describe())